## 0. Imports

In [7]:
import os
import re
import glob
import re
from google.colab import files

import pandas as pd
from typing import Dict, List, Tuple, Set

## 1. Helper functions for computing metrics to evaluate accuracy

Brief justification of selected metrics:

- **Structural Hamming Distance (SHD)**: We use SHD because it directly measures how many edge edits are needed to transform the reconstructed directed graph into the ground-truth graph: each missing true edge or extra predicted edge contributes one unit (i.e., $\mathrm{SHD} = \lvert E_{\mathrm{true}} \,\Delta\, E_{\mathrm{pred}} \rvert,$ where $\Delta$ denotes the symmetric difference of edge sets). This makes the error magnitude immediately interpretable as a count of structural mistakes, and it aligns well with the goal of assessing how dataset characteristics and scoring functions (MDL vs BDE) affect the accuracy of inferred network connectivity.

- **Jaccard distance on directed edge sets**: We additionally use Jaccard distance because it provides a normalized measure of dissimilarity between the predicted and true edge sets: $d_J = 1 - \frac{\lvert E_{\mathrm{true}}\cap E_{\mathrm{pred}} \rvert}{\lvert E_{\mathrm{true}}\cup E_{\mathrm{pred}} \rvert}.$ Unlike SHD, it is bounded in $[0,1]$, so results are easier to compare across conditions that change the number of predicted edges (e.g., different sampling/trajectory settings and different scoring functions). This complements SHD by evaluating overlap proportion rather than absolute edit count.

Together, SHD (absolute edge-edit error) and Jaccard distance (normalized edge-set overlap) provide complementary views of structural reconstruction quality across datasets and scoring functions.

In [45]:
Edge = Tuple[str, str]  # (src, dst)

In [46]:
def shd(e_true: Set[Edge], e_pred: Set[Edge]) -> int:
    """Structural Hamming Distance = |E_true Δ E_pred|."""
    return len(e_true.symmetric_difference(e_pred))

def jaccard_distance(e_true: Set[Edge], e_pred: Set[Edge]) -> float:
    """1 - |intersection|/|union| on directed edge sets."""
    union = e_true | e_pred
    if not union:
        return 0.0
    inter = e_true & e_pred
    return 1.0 - (len(inter) / len(union))

def precision_recall_f1(e_true: Set[Edge], e_pred: Set[Edge]):
    tp = len(e_true & e_pred)
    fp = len(e_pred - e_true)
    fn = len(e_true - e_pred)
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0
    return precision, recall, f1

In [47]:
def read_sif_edges(path: str) -> List[Tuple[str, float, str]]:
    """
    Read SIF lines of the form: src <whitespace> prob <whitespace> dst
    Returns list of (src, prob, dst).
    """
    rows = []
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = re.split(r"\s+", line)
            if len(parts) < 3:
                continue
            src, w, dst = parts[0], parts[1], parts[2]
            try:
                p = float(w)
            except ValueError:
                continue
            rows.append((src, p, dst))
    return rows

def binarize_edges(rows: List[Tuple[str, float, str]], method: str = "topk",
                   threshold: float = 0.5, topk: int = 3) -> Set[Edge]:
    """
    Convert weighted edges into a binary directed edge set.
    - threshold: keep edges with prob >= threshold
    - topk: for each destination node, keep top-k incoming edges by prob
    """
    if method == "threshold":
        return {(src, dst) for (src, p, dst) in rows if p >= threshold}

    if method == "topk":
        by_dst: Dict[str, List[Tuple[str, float]]] = {}
        for src, p, dst in rows:
            by_dst.setdefault(dst, []).append((src, p))

        edges: Set[Edge] = set()
        for dst, incoming in by_dst.items():
            incoming_sorted = sorted(incoming, key=lambda t: t[1], reverse=True)
            for src, _p in incoming_sorted[:topk]:
                edges.add((src, dst))
        return edges

    raise ValueError("method must be 'threshold' or 'topk'")

In [48]:
def gt_edges_as_strings(n_nodes: int, gt_map: Dict[int, List[Tuple[int, int]]]) -> Set[Edge]:
    """Convert integer edges like (2,3) to ('G2','G3')."""
    return {(f"G{u}", f"G{v}") for (u, v) in gt_map[n_nodes]}

## 2. Pipeline building a "manifest dataframe" from files in gridsearches.zip

The following code builds a DataFrame where each row corresponds to a .sif file, with parsed metadata.

In [49]:
def collect_all_sif_paths(root: str) -> list[str]:
    """
    Collect all .sif files under:
      root/gridsearch1/results MDL/
      root/gridsearch1/results BDE/
      root/gridsearch2/results MDL/
      root/gridsearch2/results BDE/
    """
    patterns = [
        os.path.join(root, "gridsearch1", "results MDL", "*.sif"),
        os.path.join(root, "gridsearch1", "results BDE", "*.sif"),
        os.path.join(root, "gridsearch2", "results MDL", "*.sif"),
        os.path.join(root, "gridsearch2", "results BDE", "*.sif"),
    ]
    paths: list[str] = []
    for pat in patterns:
        paths.extend(glob.glob(pat, recursive=True))
    return sorted(set(paths))

def parse_metadata_from_path_and_filename(path: str) -> dict:
    """
    Parse metadata from:
    - folder path: gridsearch (1/2) and score (MDL/BDE)
    - filename:
        gridsearch1: <mode>_<traj_len>_<attr_flag>_<score>.sif
                    e.g. s_50_1_MDL.sif
          rules: mode a/s, traj_len 10/20/50, attr_flag 0/1
        gridsearch2: <n_nodes>_<mode>_<step>_<score>.sif
                    e.g. 5_s_3_MDL.sif
          rules: n_nodes 5/10/16, mode a/s, step 1/2/3
    """
    norm = path.replace("\\", "/")
    filename = os.path.basename(path)

    # gridsearch inferred from path
    if "/gridsearch1/" in norm:
        gridsearch = "gridsearch1"
    elif "/gridsearch2/" in norm:
        gridsearch = "gridsearch2"
    else:
        gridsearch = None

    # score inferred from path folder
    if "/results MDL/" in norm:
        score = "MDL"
    elif "/results BDE/" in norm:
        score = "BDE"
    else:
        score = None

    meta = {
        "gridsearch": gridsearch,
        "score": score,
        "file": filename,
        "path": path,
        "traj_len": None,
        "attr_flag": None,
        "n_nodes": None,
        "step": None,
        "mode": None,
    }

    # gridsearch1 filename pattern: s_50_1_MDL.sif
    m1 = re.match(
        r"^(?P<mode>[asAS])_(?P<traj_len>\d+)_(?P<attr_flag>[01])_(?P<score>[A-Za-z]+)\.sif$",
        filename
    )
    if m1:
        meta["mode"] = m1.group("mode").lower()
        meta["traj_len"] = int(m1.group("traj_len"))
        meta["attr_flag"] = int(m1.group("attr_flag"))
        meta["score"] = meta["score"] or m1.group("score").upper()
        # gridsearch1 always uses the 5-node BN
        meta["n_nodes"] = 5
        return meta

    # gridsearch2 filename pattern: 5_s_3_MDL.sif
    m2 = re.match(
        r"^(?P<n_nodes>\d+)_(?P<mode>[asAS])_(?P<step>[123])_(?P<score>[A-Za-z]+)\.sif$",
        filename
    )
    if m2:
        meta["n_nodes"] = int(m2.group("n_nodes"))
        meta["mode"] = m2.group("mode").lower()
        meta["step"] = int(m2.group("step"))
        meta["score"] = meta["score"] or m2.group("score").upper()
        return meta

    # Fallback: infer score from filename if not found in folder
    if meta["score"] is None:
        up = filename.upper()
        if "MDL" in up:
            meta["score"] = "MDL"
        elif "BDE" in up:
            meta["score"] = "BDE"

    return meta

def build_manifest_dataframe(root: str) -> pd.DataFrame:
    """
    Build a DataFrame where each row corresponds to a .sif file,
    with parsed metadata.
    """
    paths = collect_all_sif_paths(root)
    if not paths:
        raise FileNotFoundError(
            f"No .sif files found under: {root}. Check the expected folder structure."
        )

    records = [parse_metadata_from_path_and_filename(p) for p in paths]
    df = pd.DataFrame(records)

    preferred_cols = [
        "gridsearch", "score", "n_nodes", "mode", "traj_len", "attr_flag", "step",
        "file", "path"
    ]
    cols = [c for c in preferred_cols if c in df.columns]
    df = df[cols].sort_values(
        ["gridsearch", "score", "n_nodes", "mode", "traj_len", "attr_flag", "step", "file"],
        na_position="last"
    ).reset_index(drop=True)

    return df

## 3. Loading gridsearches_loops.zip and creating a final dataframe out of all files

Assume we have access to "gridsearches_loops.zip"

In [57]:
_ = files.upload()

Saving gridsearches_loops.zip to gridsearches_loops (1).zip


In [63]:
!unzip -o gridsearches_loops.zip -d . > /dev/null

In [64]:
root = "gridsearches_loops"
manifest_df = build_manifest_dataframe(root)
manifest_df.head()

,gridsearch,score,n_nodes,mode,traj_len,attr_flag,step,file,path
0,gridsearch1,BDE,5,a,10.0,0.0,NaN,a_10_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_1...
1,gridsearch1,BDE,5,a,10.0,1.0,NaN,a_10_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_1...
2,gridsearch1,BDE,5,a,20.0,0.0,NaN,a_20_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_2...
3,gridsearch1,BDE,5,a,20.0,1.0,NaN,a_20_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_2...
4,gridsearch1,BDE,5,a,50.0,0.0,NaN,a_50_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_5...


## 4. Results for reconstructed networks in Part I of project:

In [65]:
# Ground-truth edge lists as tuples (src, dst) using integer node indices for gridsearches
GT_MAP_gridsearches = {
    5:  [(2, 2), (4, 3), (3, 4)],
    10: [(1, 1), (2, 1), (6, 1),
         (1, 2), (3, 2), (4, 2),
         (2, 3), (5, 3),
         (5, 4), (6, 4), (8, 4),
         (0, 5), (7, 5), (8, 5),
         (1, 6), (8, 6), (9, 6),
         (1, 9), (4, 9), (6, 9)],
    16: [(8, 0), (13, 0),
         (2, 2),
         (2, 3), (5, 3), (11, 3),
         (1, 4),
         (4, 5), (11, 5), (14, 5),
         (15, 6),
         (1, 8),
         (2, 9), (6, 9), (14, 9),
         (4, 10),
         (0, 11), (4, 11),
         (12, 12),
         (9, 13),
         (4, 15), (5, 15), (14, 15)],
}

In [66]:
def evaluate_manifest(manifest_df: pd.DataFrame, method: str = "topk",
                      threshold: float = 0.5, topk: int = 3) -> pd.DataFrame:
    """
    For each row in manifest_df:
    - select ground truth using n_nodes
    - read the .sif file
    - binarize predicted edges
    - compute SHD + Jaccard distance (+ precision/recall/F1)
    Returns a new DataFrame with metrics added.
    """
    records = []
    for _, row in manifest_df.iterrows():
        path = row["path"]
        n_nodes = int(row["n_nodes"])  # should be 5/10/16

        e_true = gt_edges_as_strings(n_nodes, GT_MAP_gridsearches)

        rows = read_sif_edges(path)
        e_pred = binarize_edges(rows, method=method, threshold=threshold, topk=topk)

        shd_val = shd(e_true, e_pred)
        jac_val = jaccard_distance(e_true, e_pred)
        prec, rec, f1 = precision_recall_f1(e_true, e_pred)

        records.append({
            "SHD": shd_val,
            "JaccardDist": jac_val,
            "Precision": prec,
            "Recall": rec,
            "F1": f1,
            "n_true_edges": len(e_true),
            "n_pred_edges": len(e_pred),
        })

    metrics_df = pd.DataFrame(records)
    out = pd.concat([manifest_df.reset_index(drop=True), metrics_df], axis=1)

    sort_cols = ["gridsearch", "n_nodes", "score", "mode", "traj_len", "attr_flag", "step", "file"]
    sort_cols = [c for c in sort_cols if c in out.columns]
    out = out.sort_values(sort_cols, na_position="last").reset_index(drop=True)

    out["binarize_method"] = method
    out["threshold"] = threshold if method == "threshold" else None
    out["topk"] = topk if method == "topk" else None
    return out

In [67]:
results_df = evaluate_manifest(manifest_df, method="threshold", threshold=0.5)
results_df

,gridsearch,score,n_nodes,mode,traj_len,attr_flag,step,file,path,SHD,JaccardDist,Precision,Recall,F1,n_true_edges,n_pred_edges,binarize_method,threshold,topk
0,gridsearch1,BDE,5,a,10.0,0.0,NaN,a_10_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_1...,4,0.571429,0.428571,1.000000,0.600000,3,7,threshold,0.5,None
1,gridsearch1,BDE,5,a,10.0,1.0,NaN,a_10_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_1...,4,0.571429,0.428571,1.000000,0.600000,3,7,threshold,0.5,None
2,gridsearch1,BDE,5,a,20.0,0.0,NaN,a_20_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_2...,4,0.571429,0.428571,1.000000,0.600000,3,7,threshold,0.5,None
3,gridsearch1,BDE,5,a,20.0,1.0,NaN,a_20_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_2...,4,0.571429,0.428571,1.000000,0.600000,3,7,threshold,0.5,None
4,gridsearch1,BDE,5,a,50.0,0.0,NaN,a_50_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_5...,4,0.571429,0.428571,1.000000,0.600000,3,7,threshold,0.5,None
5,gridsearch1,BDE,5,a,50.0,1.0,NaN,a_50_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/a_5...,4,0.571429,0.428571,1.000000,0.600000,3,7,threshold,0.5,None
6,gridsearch1,BDE,5,s,10.0,0.0,NaN,s_10_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/s_1...,0,0.000000,1.000000,1.000000,1.000000,3,3,threshold,0.5,None
7,gridsearch1,BDE,5,s,10.0,1.0,NaN,s_10_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/s_1...,0,0.000000,1.000000,1.000000,1.000000,3,3,threshold,0.5,None
8,gridsearch1,BDE,5,s,20.0,0.0,NaN,s_20_0_BDE.sif,gridsearches_loops/gridsearch1/results BDE/s_2...,0,0.000000,1.000000,1.000000,1.000000,3,3,threshold,0.5,None
9,gridsearch1,BDE,5,s,20.0,1.0,NaN,s_20_1_BDE.sif,gridsearches_loops/gridsearch1/results BDE/s_2...,0,0.000000,1.000000,1.000000,1.000000,3,3,threshold,0.5,None


For nicer and clearer formatting we will divide this table into one for gridsearch1 and one for gridsearch2

### Gridsearch 1 results:

In [68]:
def pivot_gs1_mdl_vs_bde(results_df: pd.DataFrame) -> pd.DataFrame:
    gs1 = results_df[results_df["gridsearch"] == "gridsearch1"].copy()

    summary = (
        gs1.groupby(["mode", "traj_len", "attr_flag", "score"], dropna=False)
           .agg(
               SHD=("SHD", "mean"),
               Jaccard=("JaccardDist", "mean"),
               Precision=("Precision", "mean"),
               Recall=("Recall", "mean"),
               F1=("F1", "mean"),
           )
           .reset_index()
    )

    pivot = summary.pivot_table(
        index=["mode", "traj_len", "attr_flag"],
        columns="score",
        values=["SHD", "Jaccard", "Precision", "Recall", "F1"],
        aggfunc="mean"
    ).sort_index()

    # Enforce metric order
    metric_order = ["SHD", "Jaccard", "Precision", "Recall", "F1"]
    pivot = pivot.reindex(metric_order, axis=1, level=0)

    # Flatten columns into e.g. "SHD (MDL)"
    pivot.columns = [f"{metric} ({score})" for (metric, score) in pivot.columns]
    return pivot.round(4)

gs1_pivot = pivot_gs1_mdl_vs_bde(results_df)
gs1_pivot

SHD (BDE)  SHD (MDL)  Jaccard (BDE)  Jaccard (MDL)  \
mode traj_len attr_flag                                                       
a    10.0     0.0              4.0        4.0         0.5714         0.5714   
              1.0              4.0        4.0         0.5714         0.5714   
     20.0     0.0              4.0        4.0         0.5714         0.5714   
              1.0              4.0        4.0         0.5714         0.5714   
     50.0     0.0              4.0        4.0         0.5714         0.5714   
              1.0              4.0        4.0         0.5714         0.5714   
s    10.0     0.0              0.0        0.0         0.0000         0.0000   
              1.0              0.0        0.0         0.0000         0.0000   
     20.0     0.0              0.0        0.0         0.0000         0.0000   
              1.0              0.0        0.0         0.0000         0.0000   
     50.0     0.0              0.0        0.0         0.0000         0.0000   
              1.0              0.0        0.0         0.0000         0.0000   

                         Precision (BDE)  Precision (MDL)  Recall (BDE)  \
mode traj_len attr_flag                                                   
a    10.0     0.0                 0.4286           0.4286           1.0   
              1.0                 0.4286           0.4286           1.0   
     20.0     0.0                 0.4286           0.4286           1.0   
              1.0                 0.4286           0.4286           1.0   
     50.0     0.0                 0.4286           0.4286           1.0   
              1.0                 0.4286           0.4286           1.0   
s    10.0     0.0                 1.0000           1.0000           1.0   
              1.0                 1.0000           1.0000           1.0   
     20.0     0.0                 1.0000           1.0000           1.0   
              1.0                 1.0000           1.0000           1.0   
     50.0     0.0                 1.0000           1.0000           1.0   
              1.0                 1.0000           1.0000           1.0   

                         Recall (MDL)  F1 (BDE)  F1 (MDL)  
mode traj_len attr_flag                                    
a    10.0     0.0                 1.0       0.6       0.6  
              1.0                 1.0       0.6       0.6  
     20.0     0.0                 1.0       0.6       0.6  
              1.0                 1.0       0.6       0.6  
     50.0     0.0                 1.0       0.6       0.6  
              1.0                 1.0       0.6       0.6  
s    10.0     0.0                 1.0       1.0       1.0  
              1.0                 1.0       1.0       1.0  
     20.0     0.0                 1.0       1.0       1.0  
              1.0                 1.0       1.0       1.0  
     50.0     0.0                 1.0       1.0       1.0  
              1.0                 1.0       1.0       1.0

### Gridsearch 2 results:

In [69]:
def pivot_gs2_mdl_vs_bde(results_df: pd.DataFrame) -> pd.DataFrame:
    gs2 = results_df[results_df["gridsearch"] == "gridsearch2"].copy()

    summary = (
        gs2.groupby(["n_nodes", "mode", "step", "score"], dropna=False)
           .agg(
               SHD=("SHD", "mean"),
               Jaccard=("JaccardDist", "mean"),
               Precision=("Precision", "mean"),
               Recall=("Recall", "mean"),
               F1=("F1", "mean"),
           )
           .reset_index()
    )

    pivot = summary.pivot_table(
        index=["n_nodes", "mode", "step"],
        columns="score",
        values=["SHD", "Jaccard", "Precision", "Recall", "F1"],
        aggfunc="mean"
    ).sort_index()

    metric_order = ["SHD", "Jaccard", "Precision", "Recall", "F1"]
    pivot = pivot.reindex(metric_order, axis=1, level=0)

    pivot.columns = [f"{metric} ({score})" for (metric, score) in pivot.columns]
    return pivot.round(4)

gs2_pivot = pivot_gs2_mdl_vs_bde(results_df)
gs2_pivot

SHD (BDE)  SHD (MDL)  Jaccard (BDE)  Jaccard (MDL)  \
n_nodes mode step                                                       
5       a    1.0         4.0        4.0         0.5714         0.5714   
             2.0         5.0        5.0         0.7143         0.7143   
             3.0         3.0        3.0         0.6000         0.6000   
        s    1.0         0.0        0.0         0.0000         0.0000   
             2.0         4.0        4.0         0.8000         0.8000   
             3.0         0.0        0.0         0.0000         0.0000   
10      a    1.0        27.0       28.0         0.9000         0.9032   
             2.0        19.0       19.0         0.7600         0.7600   
             3.0        27.0       27.0         0.9310         0.9310   
        s    1.0        10.0       10.0         0.5000         0.5000   
             2.0        20.0       21.0         0.9091         0.9130   
             3.0        20.0       20.0         0.9524         0.9524   
16      a    1.0        33.0       32.0         0.8919         0.8649   
             2.0        32.0       33.0         0.8421         0.8684   
             3.0        33.0       31.0         0.8684         0.8378   
        s    1.0        11.0       10.0         0.4783         0.4348   
             2.0        27.0       27.0         0.9310         0.9310   
             3.0        21.0       21.0         0.8750         0.8750   

                   Precision (BDE)  Precision (MDL)  Recall (BDE)  \
n_nodes mode step                                                   
5       a    1.0            0.4286           0.4286        1.0000   
             2.0            0.3333           0.3333        0.6667   
             3.0            0.5000           0.5000        0.6667   
        s    1.0            1.0000           1.0000        1.0000   
             2.0            0.3333           0.3333        0.3333   
             3.0            1.0000           1.0000        1.0000   
10      a    1.0            0.2308           0.2143        0.1500   
             2.0            0.5455           0.5455        0.3000   
             3.0            0.1818           0.1818        0.1000   
        s    1.0            1.0000           1.0000        0.5000   
             2.0            0.5000           0.4000        0.1000   
             3.0            0.5000           0.5000        0.0500   
16      a    1.0            0.2222           0.2632        0.1739   
             2.0            0.2857           0.2500        0.2609   
             3.0            0.2500           0.3000        0.2174   
        s    1.0            1.0000           1.0000        0.5217   
             2.0            0.2500           0.2500        0.0870   
             3.0            0.7500           0.7500        0.1304   

                   Recall (MDL)  F1 (BDE)  F1 (MDL)  
n_nodes mode step                                    
5       a    1.0         1.0000    0.6000    0.6000  
             2.0         0.6667    0.4444    0.4444  
             3.0         0.6667    0.5714    0.5714  
        s    1.0         1.0000    1.0000    1.0000  
             2.0         0.3333    0.3333    0.3333  
             3.0         1.0000    1.0000    1.0000  
10      a    1.0         0.1500    0.1818    0.1765  
             2.0         0.3000    0.3871    0.3871  
             3.0         0.1000    0.1290    0.1290  
        s    1.0         0.5000    0.6667    0.6667  
             2.0         0.1000    0.1667    0.1600  
             3.0         0.0500    0.0909    0.0909  
16      a    1.0         0.2174    0.1951    0.2381  
             2.0         0.2174    0.2727    0.2326  
             3.0         0.2609    0.2326    0.2791  
        s    1.0         0.5652    0.6857    0.7222  
             2.0         0.0870    0.1290    0.1290  
             3.0         0.1304    0.2222    0.2222

## 5. Results for reconstructed network in Part II of the project:

Assume we have access to "drosophilia_bn_edges.txt" (ground truth) and "small_out_MDL.sif" (reconstruction)

In [60]:
_ = files.upload()

Saving drosophilia_bn_edges.txt to drosophilia_bn_edges (1).txt


In [59]:
_ = files.upload()

Saving small_out_MDL.sif to small_out_MDL (1).sif


In [61]:
GT_PATH = "drosophilia_bn_edges.txt"
PRED_PATH = "small_out_MDL.sif"

# Ground truth from GT_PATH:
with open(GT_PATH, "r", encoding="utf-8", errors="replace") as f:
    e_true = {
        (parts[0], parts[1])
        for line in f
        if (parts := line.strip().split()) and not line.strip().startswith("#") and len(parts) >= 2
    }

rows = read_sif_edges(PRED_PATH)
e_pred_thr = binarize_edges(rows, method="threshold", threshold=0.5)

In [62]:
def compute_metrics(e_pred, method_name, threshold=None, topk=None):
    SHD = shd(e_true, e_pred)
    J = jaccard_distance(e_true, e_pred)
    P, R, F1 = precision_recall_f1(e_true, e_pred)
    return {
        "dataset": "drosophila",
        "score": "MDL",
        "binarize_method": method_name,
        "threshold": threshold,
        "topk": topk,
        "n_true_edges": len(e_true),
        "n_pred_edges": len(e_pred),
        "SHD": SHD,
        "JaccardDist": J,
        "Precision": P,
        "Recall": R,
        "F1": F1,
    }

drosophila_results = pd.DataFrame([
    compute_metrics(e_pred_thr, method_name="threshold", threshold=0.5),
]).round(4)

drosophila_results

,dataset,score,binarize_method,threshold,topk,n_true_edges,n_pred_edges,SHD,JaccardDist,Precision,Recall,F1
0,drosophila,MDL,threshold,0.5,None,9,8,1,0.1111,1.0,0.8889,0.9412
